In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import json
import os
from main import time_format
from datetime import datetime
import shutil
from PIL import Image

In [ ]:
images_folder = 'imported_images'
logs_folder = 'imported_logs'
scripts_folder = 'imported_scripts'
runs_folder = '../Neural_Lithography'

runs = [os.path.join(runs_folder, directory) for directory in os.listdir(runs_folder) if 'single_file' in directory and os.path.isdir(os.path.join(runs_folder, directory))]
for run in runs:
    for file in os.listdir(run):
        if '.png' in file:
            shutil.copyfile(os.path.join(run, file), os.path.join(images_folder, file))
        if '.log' in file:
            shutil.copyfile(os.path.join(run, file), os.path.join(logs_folder, file))
            

def merge_images(run:str):
    folder_1 = 'imported_images'
    folder_2 = 'report_images'
    files = sorted([os.path.join(folder_1, file) for file in os.listdir(folder_1) if run in file], key = lambda x: 0 if 'train' in x else 1)
    # print(f'{run=}', end=' ')
    if len(files) != 2:
        # print(f'abort {len(files)=}')
        return False
    else:
        ...
        # print()
    images = [Image.open(x) for x in files]
    widths, heights = zip(*(i.size for i in images))

    total_width = sum(widths)
    max_height = max(heights)

    new_im = Image.new('RGB', (total_width, max_height))

    x_offset = 0
    for im in images:
        new_im.paste(im, (x_offset,0))
        x_offset += im.size[0]
    
    new_im.save(os.path.join(folder_2, f'single_file_{run}.png'))
    return True

runs_list = []
for r in runs:
    parts = r.split('_')
    r = parts[-2] + '_' +  parts[-1]
    runs_list.append(r)

runs_list.sort()

text = ''
for I, run in enumerate(runs_list):
    if merge_images(run):
        template = '''
### Run {I}: {run}
- model: {model}
- epochs: {epochs}
- learning rate: {learning_rate}
- train {loss_train}
- optimization {loss_optimization}

![](report_images/single_file_{run}.png)

        '''

        def get_index(rows, string): 
            for i in range(len(rows)):
                if string in rows[i]:
                    return i
            raise ValueError(f'{string} not found in rows of file {run}')

        with open(os.path.join(logs_folder, f'logfile_{run}.log'), 'r') as file:
            rows = file.readlines()
            model = rows[get_index(rows, 'model')].split(':')[1].replace('"', '').replace(',', '').replace(' ','').replace('\n', '')
            epochs = rows[get_index(rows, 'epoch')].split(':')[1].replace(',', '').replace(' ','').replace('\n', '')
            learning_rate = rows[get_index(rows, 'model_lr')].split(':')[1].replace(',', '').replace(' ','').replace('\n', '')
            try:
                loss_train = rows[get_index(rows, ' - train')].split(' ')[-1].replace('\n', '')
            except:
                loss_train = rows[get_index(rows, 'loss function for train')].split(' ')[-1].replace('\n', '')
            
            try:
                loss_optimization = rows[get_index(rows, ' - optim')].split(' ')[-1].replace('\n', '')
            except:
                loss_optimization = rows[get_index(rows, 'mask with loss')].split(' ')[-1].replace('\n', '')

        text += template.format(
            I=I,
            run = run,
            model = model,
            epochs = epochs,
            learning_rate = learning_rate,
            loss_train = loss_train,
            loss_optimization = loss_optimization) 
        
print(text)

# Introduction
The experiments with Neural Litho from [Neural Lithography: Close the Design-to-Manufacturing Gap in
Computational Optics with a ’Real2Sim’ Learned Photolithography
Simulator](https://arxiv.org/pdf/2309.17343) of the neural network started with experiments with the original code from this [repository](https://github.com/Neural-Litho/Neural_Lithography/tree/main). The code was required to be adapted for our case.

The main difference was, that the code from the paper was used to optimize holographic optical element using the litho digital twin and optical system to compute loss. In our case we want to optimize mask, using the litho digital twin. To do that I needed to get rid of code responsible for the optical system, conversion of stepped mask to continuous mask, which meant getting rig of softmax-gumbel trick, and also the training, that is no longer takes into account the optical loss, but the litho loss only.

The set of different experiments were performed to train the model as well as optimize the mask. As a dataset I used the masks, where transmission looked like cones, and the output resist shape simulated in Dr.Litho. Below the results of the experiments are presented.

# First attempt - working with the original code

## Training and optimization results


### Run 0: 2024-07-25_10-35-28
- model: pbl3d
- epochs: 100
- learning rate: 0.01
- train mse_loss
- optimization mse_loss

![](report_images/single_file_2024-07-25_10-35-28.png)

        
### Run 2: 2024-07-25_12-43-00
- model: pbl3d
- epochs: 100
- learning rate: 0.01
- train mse_loss
- optimization mse_loss

![](report_images/single_file_2024-07-25_12-43-00.png)

        
### Run 3: 2024-07-25_13-14-18
- model: pbl3d
- epochs: 100
- learning rate: 0.01
- train mse_loss
- optimization mse_loss

![](report_images/single_file_2024-07-25_13-14-18.png)

        
### Run 4: 2024-07-25_13-43-43
- model: pbl3d
- epochs: 100
- learning rate: 0.001
- train mse_loss
- optimization mse_loss

![](report_images/single_file_2024-07-25_13-43-43.png)

        
### Run 5: 2024-07-25_14-11-59
- model: pbl3d
- epochs: 100
- learning rate: 0.001
- train mse_loss
- optimization mse_loss

![](report_images/single_file_2024-07-25_14-11-59.png)

        
### Run 6: 2024-07-25_16-42-28
- model: pbl3d
- epochs: 100
- learning rate: 0.001
- train mse_loss
- optimization mse_loss

![](report_images/single_file_2024-07-25_16-42-28.png)

        
### Run 7: 2024-07-27_21-55-06
- model: pbl3d
- epochs: 100
- learning rate: 0.001
- train mse_loss
- optimization mse_loss

![](report_images/single_file_2024-07-27_21-55-06.png)

        
### Run 8: 2024-07-27_23-02-42
- model: pbl3d
- epochs: 100
- learning rate: 0.001
- train mse_loss
- optimization mse_loss

![](report_images/single_file_2024-07-27_23-02-42.png)

        
### Run 9: 2024-07-27_23-17-18
A promising result. What was done? 
- model: pbl3d
- epochs: 100
- learning rate: 0.001
- train mse_loss
- optimization mse_loss

![](report_images/single_file_2024-07-27_23-17-18.png)

        
### Run 10: 2024-07-28_00-02-26
- model: pbl3d
- epochs: 100
- learning rate: 0.001
- train mse_loss
- optimization mse_loss

![](report_images/single_file_2024-07-28_00-02-26.png)

        
### Run 11: 2024-07-28_00-27-20
- model: fno
- epochs: 100
- learning rate: 0.001
- train mse_loss
- optimization mse_loss

![](report_images/single_file_2024-07-28_00-27-20.png)

        
### Run 12: 2024-07-28_00-55-10
- model: pbl3d
- epochs: 100
- learning rate: 0.001
- train mse_loss
- optimization mse_loss

![](report_images/single_file_2024-07-28_00-55-10.png)

        
### Run 14: 2024-07-28_01-23-31
- model: pbl3d
- epochs: 100
- learning rate: 0.001
- train mse_loss
- optimization mse_loss

![](report_images/single_file_2024-07-28_01-23-31.png)

        
### Run 18: 2024-07-28_13-50-25
- model: pbl3d
- epochs: 100
- learning rate: 0.01
- train mse_loss
- optimization mse_loss

![](report_images/single_file_2024-07-28_13-50-25.png)

        
### Run 19: 2024-07-28_13-55-01
- model: pbl3d
- epochs: 100
- learning rate: 0.001
- train mse_loss
- optimization mse_loss

![](report_images/single_file_2024-07-28_13-55-01.png)

        
### Run 20: 2024-07-28_14-25-24
- model: pbl3d
- epochs: 100
- learning rate: 0.001
- train mse_loss
- optimization mse_loss

![](report_images/single_file_2024-07-28_14-25-24.png)

        
### Run 23: 2024-07-28_17-01-55
- model: pbl3d
- epochs: 100
- learning rate: 0.0001
- train mse_loss
- optimization mse_loss

![](report_images/single_file_2024-07-28_17-01-55.png)

        
### Run 24: 2024-07-28_17-08-24
- model: pbl3d
- epochs: 100
- learning rate: 0.01
- train mse_loss
- optimization mse_loss

![](report_images/single_file_2024-07-28_17-08-24.png)

        


# Second attempt - Creating MLOps pipeline and creating own models

In [6]:
folder = 'model_runs'
valid_runs = []
for run in os.listdir(folder):
    run_folder = os.path.join(folder, run)
    if len([x for x in os.listdir(run_folder) if '.png' in x]) > 1:
        valid_runs.append(run_folder)

valid_runs.sort()
# print(*valid_runs, sep='\n')

def json_to_md(js:dict):
    text = ''
    for key, val in js.items():
        text += f'- {key}: {val}\n'

    return text

def merge_images(run:str):
    folder_1 = 'model_runs'
    folder_2 = 'report_image_2'
    files = [file for file in os.listdir(os.path.join(folder_1, run)) if '.png' in file]
    files = sorted(files, key=lambda x: 0 if 'training' in x else 1)
    images = [Image.open(x) for x in files]
    widths, heights = zip(*(i.size for i in images))

    total_width = sum(widths)
    max_height = max(heights)

    new_im = Image.new('RGB', (total_width, max_height))

    x_offset = 0
    for im in images:
        new_im.paste(im, (x_offset,0))
        x_offset += im.size[0]
    
    new_im.save(os.path.join(folder_2, f'{run}.png'))
    return True

template = '''
### Run {I}: {run}

Training parameters:
{training_parameters}

![](model_runs/run_{run}/training.png)

Optimization parameters:
{optimization_parameters}

![](model_runs/run_{run}/optimization.png)

'''

text = ''
for I, run in enumerate(valid_runs):
    with open(os.path.join(run, 'results.json'), 'r') as file:
        js = json.load(file)

    txt = template.format(
        I = I,
        run = js['time'],
        training_parameters = json_to_md(js['training']['parameters']),
        optimization_parameters = json_to_md(js['optimization']['parameters'])
        )
    text += txt
    # break
print(text)


### Run 0: 2024.07.28_22:19:07

Training parameters:
- plotting_interval: 10
- batch_size: 10
- split_percent: 0.1
- shuffle_dataset: False
- learning_rate: 0.0001
- num_epochs: 1000
- early_stopping_patience: 5
- optimizer: <class 'torch.optim.adam.Adam'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.28_22:19:07/training.png)

Optimization parameters:
- target_mask_fname: ../Neural_Lithography/data/printed_data/mask/data_0.npy
- target_dart_fname: ../Neural_Lithography/data/printed_data/afm/data_0.npy
- plotting_interval: 100
- learning_rate: 0.1
- num_epochs: 1000
- optimizer: <class 'torch.optim.sgd.SGD'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.28_22:19:07/optimization.png)


### Run 1: 2024.07.28_22:51:07

Training parameters:
- plotting_interval: 10
- batch_size: 10
- split_percent: 0.1
- shuffle_dataset: False
- learning_rate: 0.0001
- num_epochs: 1000
- early_stopping_patience: 5
- optimizer: <class 'torch.optim.adam.Adam'>
- loss_function: MSELoss()




### Run 0: 2024.07.28_22:19:07

Training parameters:
- plotting_interval: 10
- batch_size: 10
- split_percent: 0.1
- shuffle_dataset: False
- learning_rate: 0.0001
- num_epochs: 1000
- early_stopping_patience: 5
- optimizer: <class 'torch.optim.adam.Adam'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.28_22:19:07/training.png)

Optimization parameters:
- target_mask_fname: ../Neural_Lithography/data/printed_data/mask/data_0.npy
- target_dart_fname: ../Neural_Lithography/data/printed_data/afm/data_0.npy
- plotting_interval: 100
- learning_rate: 0.1
- num_epochs: 1000
- optimizer: <class 'torch.optim.sgd.SGD'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.28_22:19:07/optimization.png)


### Run 1: 2024.07.28_22:51:07

Training parameters:
- plotting_interval: 10
- batch_size: 10
- split_percent: 0.1
- shuffle_dataset: False
- learning_rate: 0.0001
- num_epochs: 1000
- early_stopping_patience: 5
- optimizer: <class 'torch.optim.adam.Adam'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.28_22:51:07/training.png)

Optimization parameters:
- target_mask_fname: ../Neural_Lithography/data/printed_data/mask/data_0.npy
- target_dart_fname: ../Neural_Lithography/data/printed_data/afm/data_0.npy
- plotting_interval: 100
- learning_rate: 0.1
- num_epochs: 1000
- optimizer: <class 'torch.optim.sgd.SGD'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.28_22:51:07/optimization.png)


### Run 2: 2024.07.28_23:50:48

Training parameters:
- plotting_interval: 10
- batch_size: 50
- split_percent: 0.1
- shuffle_dataset: False
- learning_rate: 0.0001
- num_epochs: 1000
- early_stopping_patience: 5
- optimizer: <class 'torch.optim.adam.Adam'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.28_23:50:48/training.png)

Optimization parameters:
- target_mask_fname: ../Neural_Lithography/data/printed_data/mask/data_0.npy
- target_dart_fname: ../Neural_Lithography/data/printed_data/afm/data_0.npy
- plotting_interval: 100
- learning_rate: 0.1
- num_epochs: 1000
- optimizer: <class 'torch.optim.sgd.SGD'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.28_23:50:48/optimization.png)


### Run 3: 2024.07.29_00:00:37

Training parameters:
- plotting_interval: 10
- batch_size: 50
- split_percent: 0.1
- shuffle_dataset: False
- learning_rate: 0.0001
- num_epochs: 1000
- early_stopping_patience: 5
- optimizer: <class 'torch.optim.adam.Adam'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.29_00:00:37/training.png)

Optimization parameters:
- target_mask_fname: ../Neural_Lithography/data/printed_data/mask/data_0.npy
- target_dart_fname: ../Neural_Lithography/data/printed_data/afm/data_0.npy
- plotting_interval: 100
- learning_rate: 0.1
- num_epochs: 1000
- optimizer: <class 'torch.optim.sgd.SGD'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.29_00:00:37/optimization.png)


### Run 4: 2024.07.29_00:27:02

Training parameters:
- plotting_interval: 10
- batch_size: 50
- split_percent: 0.1
- shuffle_dataset: False
- learning_rate: 0.0001
- num_epochs: 1000
- early_stopping_patience: 5
- optimizer: <class 'torch.optim.adam.Adam'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.29_00:27:02/training.png)

Optimization parameters:
- target_mask_fname: ../Neural_Lithography/data/printed_data/mask/data_0.npy
- target_dart_fname: ../Neural_Lithography/data/printed_data/afm/data_0.npy
- plotting_interval: 100
- learning_rate: 0.1
- num_epochs: 1000
- optimizer: <class 'torch.optim.sgd.SGD'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.29_00:27:02/optimization.png)


### Run 5: 2024.07.29_00:36:05

Training parameters:
- plotting_interval: 10
- batch_size: 50
- split_percent: 0.1
- shuffle_dataset: False
- learning_rate: 0.0001
- num_epochs: 1000
- early_stopping_patience: 5
- optimizer: <class 'torch.optim.adam.Adam'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.29_00:36:05/training.png)

Optimization parameters:
- target_mask_fname: ../Neural_Lithography/data/printed_data/mask/data_0.npy
- target_dart_fname: ../Neural_Lithography/data/printed_data/afm/data_0.npy
- plotting_interval: 100
- learning_rate: 0.1
- num_epochs: 1000
- optimizer: <class 'torch.optim.sgd.SGD'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.29_00:36:05/optimization.png)


### Run 6: 2024.07.29_00:43:14

Training parameters:
- plotting_interval: 10
- batch_size: 50
- split_percent: 0.1
- shuffle_dataset: False
- learning_rate: 0.0001
- num_epochs: 1000
- early_stopping_patience: 5
- optimizer: <class 'torch.optim.adam.Adam'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.29_00:43:14/training.png)

Optimization parameters:
- target_mask_fname: ../Neural_Lithography/data/printed_data/mask/data_0.npy
- target_dart_fname: ../Neural_Lithography/data/printed_data/afm/data_0.npy
- plotting_interval: 100
- learning_rate: 10
- num_epochs: 1000
- optimizer: <class 'torch.optim.sgd.SGD'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.29_00:43:14/optimization.png)


### Run 7: 2024.07.29_00:45:40

Training parameters:
- plotting_interval: 10
- batch_size: 50
- split_percent: 0.1
- shuffle_dataset: False
- learning_rate: 0.0001
- num_epochs: 1000
- early_stopping_patience: 5
- optimizer: <class 'torch.optim.adam.Adam'>
- loss_function: BCELoss()


![](model_runs/run_2024.07.29_00:45:40/training.png)

Optimization parameters:
- target_mask_fname: ../Neural_Lithography/data/printed_data/mask/data_0.npy
- target_dart_fname: ../Neural_Lithography/data/printed_data/afm/data_0.npy
- plotting_interval: 100
- learning_rate: 10
- num_epochs: 1000
- optimizer: <class 'torch.optim.sgd.SGD'>
- loss_function: BCELoss()


![](model_runs/run_2024.07.29_00:45:40/optimization.png)


### Run 8: 2024.07.29_11:21:05

Training parameters:
- plotting_interval: 10
- batch_size: 50
- split_percent: 0.1
- shuffle_dataset: False
- learning_rate: 0.0001
- num_epochs: 1000
- early_stopping_patience: 5
- optimizer: <class 'torch.optim.adam.Adam'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.29_11:21:05/training.png)

Optimization parameters:
- target_mask_fname: ../Neural_Lithography/data/printed_data/mask/data_0.npy
- target_dart_fname: ../Neural_Lithography/data/printed_data/afm/data_0.npy
- plotting_interval: 100
- learning_rate: 10
- num_epochs: 1000
- optimizer: <class 'torch.optim.sgd.SGD'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.29_11:21:05/optimization.png)


### Run 9: 2024.07.29_11:25:25

Training parameters:
- plotting_interval: 1
- batch_size: 10
- split_percent: 0.1
- shuffle_dataset: False
- learning_rate: 0.0001
- num_epochs: 1000
- early_stopping_patience: 5
- optimizer: <class 'torch.optim.adam.Adam'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.29_11:25:25/training.png)

Optimization parameters:
- target_mask_fname: ../Neural_Lithography/data/printed_data/mask/data_0.npy
- target_dart_fname: ../Neural_Lithography/data/printed_data/afm/data_0.npy
- plotting_interval: 100
- learning_rate: 10
- num_epochs: 1000
- optimizer: <class 'torch.optim.sgd.SGD'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.29_11:25:25/optimization.png)


### Run 10: 2024.07.29_15:13:44

Training parameters:
- plotting_interval: 1
- batch_size: 10
- split_percent: 0.1
- shuffle_dataset: False
- learning_rate: 0.0001
- num_epochs: 1000
- early_stopping_patience: 5
- optimizer: <class 'torch.optim.adam.Adam'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.29_15:13:44/training.png)

Optimization parameters:
- target_mask_fname: ../Neural_Lithography/data/printed_data/mask/data_0.npy
- target_dart_fname: ../Neural_Lithography/data/printed_data/afm/data_0.npy
- plotting_interval: 100
- learning_rate: 10
- num_epochs: 10000
- optimizer: <class 'torch.optim.sgd.SGD'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.29_15:13:44/optimization.png)


### Run 11: 2024.07.29_18:38:41

Training parameters:
- plotting_interval: 1
- batch_size: 10
- split_percent: 0.1
- shuffle_dataset: False
- learning_rate: 0.0001
- num_epochs: 1000
- early_stopping_patience: 5
- optimizer: <class 'torch.optim.adam.Adam'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.29_18:38:41/training.png)

Optimization parameters:
- target_mask_fname: ../Neural_Lithography/data/printed_data/mask/data_0.npy
- target_dart_fname: ../Neural_Lithography/data/printed_data/afm/data_0.npy
- plotting_interval: 100
- learning_rate: 1
- num_epochs: 10000
- optimizer: <class 'torch.optim.sgd.SGD'>
- loss_function: MSELoss()


![](model_runs/run_2024.07.29_18:38:41/optimization.png)




# Analysis and conclusion